In [1]:
import glob
import os
import pyodbc
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from dotenv import load_dotenv

Iniciando o Spark Session

In [ ]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4051') \
    .getOrCreate()

Variáveis para os diretórios dos dados brutos

In [3]:
clients = "Data/Clients"

transactions_in = "Data/Transactions-in"
transactions_out = "Data/Transactions-out"

Variáveis para os schemas dos DataFrames

In [4]:
clients_schema = StructType([
        StructField("id", IntegerType(), True),
        StructField("nome", StringType(), True),
        StructField("email", StringType(), True),
        StructField("data_cadastro", TimestampType(), True),
        StructField("telefone", StringType(), True)
    ])

transactions_schema = StructType([
        StructField("id", IntegerType(), True),
        StructField("cliente_id", IntegerType(), True),
        StructField("valor", DoubleType(), True),
        StructField("data", TimestampType(), True),
    ])

Função para carregar os arquivos .csv convertendo-os em DataFrames usando Spark

In [5]:
def transform_csv_to_df(spark, path, schema):
    if not os.path.isdir(path):
        raise ValueError(f"{path} não é um diretório válido.")

    list_paths_csv = glob.glob(os.path.join(path, '*.csv'))

    if not list_paths_csv:
        raise ValueError(f"Não foram encontrados arquivos csv em {path}.")

    df = spark.read.csv(list_paths_csv, sep=';', schema=schema, inferSchema=True)

    df = df.filter(~col('id').contains('id'))

    return df

Função para verificar se há dados nulos ou não informados nos DataFrames, caso sim, os dados são preenchidos

In [6]:
def verify_empty_data(df):
    for col_name in df.columns:
        data_type = df.schema[col_name].dataType
        if data_type == StringType():
            count_empty = df.filter((col(col_name) == '') | isnull(col_name) | isnan(col_name) | (col(col_name).isNull())).count()
            if count_empty != 0:
                print(f"Column '{col_name}' has {count_empty} empty/null/none/NaN values.")
                df = df.fillna({col_name: 'Não informado'})
        elif data_type == IntegerType():
            count_null = df.filter(col(col_name).isNull()).count()
            if count_null != 0:
                print(f"Column '{col_name}' has {count_null} null values.")
                df = df.fillna({col_name: 0})
        elif data_type == DoubleType():
            count_null = df.filter(col(col_name).isNull()).count()
            if count_null != 0:
                print(f"Column '{col_name}' has {count_null} null values.")
                df = df.fillna({col_name: 0.00})
        elif data_type == TimestampType():
            count_null = df.filter(col(col_name).isNull()).count()
            if count_null != 0:
                print(f"Column '{col_name}' has {count_null} null values.")
                df = df.fillna({col_name: '1900-01-01 00:00:00'})
    return df

Função para padronizar os dados da coluna valor dos DataFrames transactions_in e transactions_out corrigindo-os para que todos tenham duas casas decimais e valor absoluto

In [7]:
def correcting_data(df):
    df = df.withColumn("valor", round(col("valor"), 2))
    df = df.withColumn('valor', expr('abs(valor)'))
    return df

Função para criar a coluna estado no DataFrame clients a partir do número do DDD extraído da coluna telefone

In [8]:
def add_state_column(df):
    df = df.withColumn('DDD', split(df['telefone'], r'[()]+').getItem(1))
    df = df.withColumn('estado', when(col('DDD') == '20', 'Paraíba')
                        .when(col('DDD') == '21', 'Rio de Janeiro')
                        .when(col('DDD') == '22', 'Mato Grosso')
                        .when(col('DDD') == '23', 'Pernambuco')
                        .when(col('DDD') == '24', 'Rio de Janeiro')
                        .when(col('DDD') == '25', 'Bahia')
                        .when(col('DDD') == '26', 'Minas Gerais')
                        .when(col('DDD') == '27', 'Espírito Santo')
                        .when(col('DDD') == '28', 'Roraima')
                        .when(col('DDD') == '29', 'São Paulo')
                        .when(col('DDD') == '30', 'Maranhão')
                        .otherwise('Inválido'))
    df = df.drop('DDD')
    return df

Função para formatar os nomes dos clientes, dividindo em duas colunas denominadas nome e sobrenome, verificando os que não possuem sobrenome e preenchendo com 'Não informado'

In [9]:
def format_names(df):
    df = df.withColumn("nome_split", split(df.nome, " "))
    df = df.withColumn("nome", df.nome_split[0])
    df = df.withColumn("sobrenome1", df.nome_split[1])
    df = df.withColumn("sobrenome2", df.nome_split[2])
    df = df.withColumn("sobrenome3", df.nome_split[3])
    df = df.withColumn("sobrenome4", df.nome_split[4])
    df = df.withColumn("sobrenome5", df.nome_split[5])
    df = df.withColumn("sobrenome6", df.nome_split[6])
    df = df.withColumn("sobrenome", concat_ws(" ", "sobrenome1", "sobrenome2", "sobrenome3", "sobrenome4", "sobrenome5", "sobrenome6"))
    df = df.drop("nome_split", "sobrenome1", "sobrenome2", "sobrenome3", "sobrenome4", "sobrenome5", "sobrenome6")
    df = df.withColumn("nome", initcap(df.nome))
    df = df.withColumn("sobrenome", initcap(df.sobrenome))
    df = df.withColumn("sobrenome", when(df.sobrenome == "", "Não informado").otherwise(df.sobrenome))
    df = df.select("id", "nome", "sobrenome", "email", "data_cadastro", "telefone", "estado")
    return df

Função para verificar se todos ids de clientes existentes nos DataFrames de transactions_in e transactions_out correspondem a um id no DataFrame clients e caso não exista, o id é adicionado no DataFrame clients preenchendo as demais colunas com 'Não localizado'

In [10]:
def verify_client_id_existence(spark, df_transactions, df_clients):
    df_ids_transactions = df_transactions.select(col('cliente_id'))
    df_ids_clients = df_clients.select(col('id'))
    df_new_clients = df_ids_transactions.join(df_ids_clients, df_ids_transactions.cliente_id == df_ids_clients.id, "leftanti")
    df_new_clients = df_new_clients.distinct()
    df_new_clients = df_new_clients.withColumnRenamed("cliente_id", "id")
    df_new_clients = df_new_clients.withColumn('nome', lit('Não localizado'))
    df_new_clients = df_new_clients.withColumn('sobrenome', lit('Não localizado'))
    df_new_clients = df_new_clients.withColumn('email', lit('Não localizado'))
    df_new_clients = df_new_clients.withColumn('data_cadastro', lit('1900-01-01 00:00:00').cast('timestamp'))
    df_new_clients = df_new_clients.withColumn('telefone', lit('Não localizado'))
    df_new_clients = df_new_clients.withColumn('estado', lit('Não localizado'))
    df_clients = df_clients.unionAll(df_new_clients)
    return df_clients

Função para renomear as colunas do DataFrame

In [11]:
def renamed_column(df, previous_column, new_column):
     return df.withColumnRenamed(previous_column, new_column)

Função para realizar a conexão com o banco de dados SQL Server na Azure

In [12]:
def connection_database():
    load_dotenv()
    server_name = os.environ["server_name"]
    database_name = os.environ["database_name"]
    username = os.environ["username"]
    password = os.environ["password"]

    connection_string = f"Driver={{ODBC Driver 18 for SQL Server}};\
        Server=tcp:{server_name},1433;\
        Database={database_name};\
        Uid={username};\
        Pwd={password};\
        Encrypt=yes;\
        TrustServerCertificate=no;Connection Timeout=30;"
    return pyodbc.connect(connection_string)

Função para criar a tabela CLIENTS no banco de dados SQL Server na Azure

In [13]:
def create_table_clients(conn, df):
    cursor = conn.cursor()
    cursor.execute(f"SELECT COUNT(*) FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'clientes'")
    if cursor.fetchone()[0] == 0:
        create_table_query = f"CREATE TABLE clientes (\
                                    id INTEGER PRIMARY KEY,\
                                    nome VARCHAR(255),\
                                    sobrenome VARCHAR(255),\
                                    email VARCHAR(255),\
                                    data_hora_cadastro DATETIME,\
                                    telefone VARCHAR(255),\
                                    estado VARCHAR(255)\
                                    );"

        cursor.execute(create_table_query)
        conn.commit()
        print("Tabela clientes criada com sucesso!")
    else:
        print("A tabela clientes já está no banco de dados!")

Função para criar as tabelas TRANSACTIONS_IN e TRANSACTIONS_OUT no banco de dados SQL Server na Azure

In [14]:
def create_table_transactions(conn, df, name_table):
    cursor = conn.cursor()
    cursor.execute(f"SELECT COUNT(*) FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{name_table}'")
    if cursor.fetchone()[0] == 0:
        create_table_query = f"CREATE TABLE {name_table} (\
                                id INTEGER PRIMARY KEY,\
                                cliente_id INTEGER REFERENCES clientes (id),\
                                valor DECIMAL(10,2),\
                                data_hora DATETIME,\
                            );"

        cursor.execute(create_table_query)
        conn.commit()
        print(f"Tabela {name_table} criada com sucesso!")
    else: 
        print(f"A tabela {name_table} já está no banco de dados!")

Função para inserir os dataframes nas tabelas já existentes no banco de dados SQL Server na Azure

In [15]:
def insert_df_into_db(conn, df, name_table):
    try:
        cursor = conn.cursor()
        columns = ",".join(df.columns)      
        placeholders = ",".join("?" for _ in df.columns) 
        df = df.rdd.collect()

        for values in df:
            cursor = conn.cursor()
            cursor.execute(f"INSERT INTO {name_table} ({columns}) VALUES ({placeholders})", values)
            cursor.commit()
        print("Os dados foram inseridos com sucesso na tabela.")
    except pyodbc.IntegrityError:
        print(f"Os dados já existem no banco de dados!")
        conn.rollback()
    except Exception as e:
        print(f"Ocorreu um erro ao inserir os dados na tabela: {e}")
        conn.rollback()
    finally:
        cursor.close()

In [ ]:
try:
    print("Transformando os arquivos CSVs em data frames...")
    df_clients = transform_csv_to_df(spark, clients, clients_schema)
    df_transactions_in = transform_csv_to_df(spark, transactions_in, transactions_schema)
    df_transactions_out = transform_csv_to_df(spark, transactions_out , transactions_schema)
    print("OK")

    print("Verificando se há dados não informados nas colunas dos DataFrames...")
    df_clients = verify_empty_data(df_clients)
    df_transactions_in = verify_empty_data(df_transactions_in)
    df_transactions_out = verify_empty_data(df_transactions_out)
    print("OK")

    print("Corrigindo os dados da coluna valor dos DataFrames de transações...")
    df_transactions_in = correcting_data(df_transactions_in)
    df_transactions_out = correcting_data(df_transactions_out)
    print("OK")

    print("Formatando o DataFrame de clientes...")
    df_clients = add_state_column(df_clients)
    df_clients = format_names(df_clients)
    df_clients = verify_client_id_existence(spark, df_transactions_in, df_clients)
    df_clients = verify_client_id_existence(spark, df_transactions_out, df_clients)
    print("OK")
    
    print("Alterando o nome das colunas de data e hora dos DataFrames...")
    df_clients = renamed_column(df_clients,"data_cadastro", "data_hora_cadastro")
    df_transactions_in = renamed_column(df_transactions_in, "data", "data_hora")
    df_transactions_out = renamed_column(df_transactions_out, "data", "data_hora")
    print("OK")

    try:
        print("Conectando com o banco de dados...")
        conn = connection_database()
        print("OK")
    except Exception as e:
        print(f"Não foi possivel se conectar com o banco de dados! Por causa do seguinte erro: {e}")
    else:
        print("\nCriando tabela de clientes no banco de dados!")
        create_table_clients(conn, df_clients)
        
        print("\nInserindo dados na tabela...")
        insert_df_into_db(conn, df_clients, "clientes")
            
        print("\nCriando a tabela de transações in no banco de dados!")
        create_table_transactions(conn, df_transactions_in, "transactions_in")

        print("\nInserindo dados na tabela...")
        insert_df_into_db(conn, df_transactions_in, "transactions_in")
    
        print("\nCriando a tabela de transações out no banco de dados!")
        create_table_transactions(conn, df_transactions_out, "transactions_out")

        print("\nInserindo dados na tabela...")
        insert_df_into_db(conn, df_transactions_out, "transactions_out")
       
    print("\n")
    print("-" * 30)
    print("Transações in")
    df_transactions_in.show()
    print("-" * 30)
    print("Transações out")
    df_transactions_out.show()
    print("-" * 30)
    print("Dados dos clientes")
    df_clients.show()
    
except Exception as e:
    print(f"Ocorreu o seguinte erro: {e}!")